# CEO-Request Challenge

In [123]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data Preparation

In [124]:
from olist.seller import Seller
sellers = Seller().get_training_data()
sellers.head(2)

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,total_sub,n_orders,quantity,quantity_per_order,sales,sales_fees,share_of_five_stars,share_of_one_stars,review_score
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.0,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,320.0,3,3,1.000,218.70,21.870,0.333333,0.333333,3.000000
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.0,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,1120.0,40,41,1.025,11703.07,1170.307,0.731707,0.048780,4.560976


In [125]:
from olist.seller import Order
order_reviews = Order().get_review_score()
order_reviews.head(2)

,order_id,dim_is_five_star,dim_is_one_star,review_score
0,73fc7af87114b39712e6da79b0a377eb,0,0,4
1,a548910a1c6147796b98fdf73dbeba33,1,0,5


In [126]:
sellers.columns

Index(['seller_id', 'seller_city', 'seller_state', 'delay_to_carrier',
       'wait_time', 'date_first_sale', 'date_last_sale', 'months_on_olist',
       'total_sub', 'n_orders', 'quantity', 'quantity_per_order', 'sales',
       'sales_fees', 'share_of_five_stars', 'share_of_one_stars',
       'review_score'],
      dtype='object')

In [127]:
order_reviews.columns

Index(['order_id', 'dim_is_five_star', 'dim_is_one_star', 'review_score'], dtype='object')

### Revenue

In [135]:
# Computing the total revenue within the class

revenue_sel = Seller().get_revenue()
revenue_sel

,seller_id,sales_fees,total_sub,total_revenue
0,0015a82c2db000af6aaaf3ae2ecb0532,268.500,80.0,348.500
1,001cca7ae9ae17fb1caed9dfb1094831,2508.003,1360.0,3868.003
2,001e6ad469a905060d959994f1b41e4f,25.000,0.0,25.000
3,002100f778ceb8431b7a1020ff7ab48f,123.450,560.0,683.450
4,003554e2dce176b5555353e4f3555ac8,12.000,0.0,12.000
...,...,...,...,...
3090,ffcfefa19b08742c5d315f2791395ee5,6.990,0.0,6.990
3091,ffdd9f82b9a447f6f8d4b91554cc7dd3,210.120,1440.0,1650.120
3092,ffeee66ac5d5a62fe688b9d26f83f534,183.986,560.0,743.986
3093,fffd5413c0700ac820c7069d66d98c89,906.230,1040.0,1946.230


In [136]:
# Computing the total revenue in this notebook

sellers['revenue'] = sellers['sales_fees'] + sellers['total_sub']
sellers.head(3)

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,total_sub,n_orders,quantity,quantity_per_order,sales,sales_fees,share_of_five_stars,share_of_one_stars,review_score,revenue
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.0,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,320.0,3,3,1.000,218.70,21.870,0.333333,0.333333,3.000000,341.870
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.0,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,1120.0,40,41,1.025,11703.07,1170.307,0.731707,0.048780,4.560976,2290.307
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.0,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.0,0.0,1,1,1.000,158.00,15.800,1.000000,0.000000,5.000000,15.800


In [137]:
revenue = sellers[['seller_id', 'revenue']].drop_duplicates()

In [138]:
revenue

,seller_id,revenue
0,3442f8959a84dea7ee197c632cb2df15,341.870
1,d1b65fc7debc3361ea86b5f14c68d2e2,2290.307
2,ce3ad9de960102d0677a81f5d0bb7b2d,15.800
3,c0f3eea2e14555b6faeea3dd58c1b1c3,7.999
4,51a04a8a6bdcb23deccc82b0b80742cf,16.799
...,...,...
2962,98dddbc4601dd4443ca174359b237166,15.800
2963,f8201cab383e484733266d1906e2fdfa,1208.900
2964,74871d19219c7d518d0090283e03c137,455.004
2965,e603cf3fec55f8697c9059638d6c8eb5,349.700


In [139]:
from olist.data import Olist
data_df = Olist().get_data()

### Cost of Reviews

In [140]:
ids = data_df['order_items'][['order_id', 'seller_id']].drop_duplicates()
ids

,order_id,seller_id
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87
...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,b8bc237ba3788b23da09c0f1f3a3288c
112646,fffcd46ef2263f404302a634eb57f7eb,f3c38ab652836d21de61fb8314b69182
112647,fffce4705a9662cd70adb13d4a31832d,c3cfdc648177fdbbbb35635a37472c53
112648,fffe18544ffabc95dfada21779c9644f,2b3e4a2a3ea8e01938cabda2a3e5cc79


In [141]:
df = pd.merge(ids, order_reviews, on='order_id', how='inner')
df.head()

,order_id,seller_id,dim_is_five_star,dim_is_one_star,review_score
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,1,0,5
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,0,0,4
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,1,0,5
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,0,0,4
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,1,0,5


In [142]:
df['cost_of_review'] = order_reviews.review_score.map({1: 100, 2: 50, 3: 40, 4: 0, 5: 0})
df.head(10)

,order_id,seller_id,dim_is_five_star,dim_is_one_star,review_score,cost_of_review
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,1,0,5,0.0
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,0,0,4,0.0
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,1,0,5,0.0
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,0,0,4,0.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,1,0,5,0.0
5,00048cc3ae777c65dbb7d2a0634bc1ea,6426d21aca402a131fc0a5d0960a3c90,0,0,4,100.0
6,00054e8431b9d7675808bcb819fb4a32,7040e82f899a04d1b434b795a43b4617,0,0,4,0.0
7,000576fe39319847cbb9d288c5617fa6,5996cddab893a4652a15592fb58ab8db,1,0,5,0.0
8,0005a1a1728c9d785b8e2b08b904576c,a416b6a846a11724393025641d4edd5e,0,1,1,0.0
9,0005f50442cb953dcd1d21e1fb923495,ba143b05f0110f0dc71ad71b4466ce92,0,0,4,0.0


In [143]:
df = df.groupby(['seller_id']).agg({'dim_is_one_star': 'mean', 
                                    'dim_is_five_star': 'mean',
                                    'review_score': 'mean', 
                                    'cost_of_review': 'sum'}).reset_index()
df.head(3)

,seller_id,dim_is_one_star,dim_is_five_star,review_score,cost_of_review
0,0015a82c2db000af6aaaf3ae2ecb0532,0.333333,0.666667,3.666667,140.0
1,001cca7ae9ae17fb1caed9dfb1094831,0.121827,0.527919,3.984772,3560.0
2,001e6ad469a905060d959994f1b41e4f,1.000000,0.000000,1.000000,0.0


In [144]:
df.columns = ['seller_id', 'share_of_one_stars', 'share_of_five_stars', 'review_score', 'cost_of_reviews']
df.head(3)

,seller_id,share_of_one_stars,share_of_five_stars,review_score,cost_of_reviews
0,0015a82c2db000af6aaaf3ae2ecb0532,0.333333,0.666667,3.666667,140.0
1,001cca7ae9ae17fb1caed9dfb1094831,0.121827,0.527919,3.984772,3560.0
2,001e6ad469a905060d959994f1b41e4f,1.000000,0.000000,1.000000,0.0


In [145]:
final_df = pd.merge(df, revenue, on='seller_id', how='inner')
final_df.head()

,seller_id,share_of_one_stars,share_of_five_stars,review_score,cost_of_reviews,revenue
0,0015a82c2db000af6aaaf3ae2ecb0532,0.333333,0.666667,3.666667,140.0,348.500
1,001cca7ae9ae17fb1caed9dfb1094831,0.121827,0.527919,3.984772,3560.0,3868.003
2,002100f778ceb8431b7a1020ff7ab48f,0.134615,0.519231,3.903846,840.0,683.450
3,003554e2dce176b5555353e4f3555ac8,0.000000,1.000000,5.000000,0.0,12.000
4,004c9cd9d87a3c30c522c48c4fc07416,0.080745,0.596273,4.136646,2400.0,3171.271


### Profits

In [148]:
final_df['profits'] = final_df['revenue'] - final_df['cost_of_reviews']
final_df.head(3)

,seller_id,share_of_one_stars,share_of_five_stars,review_score,cost_of_reviews,revenue,profits
0,0015a82c2db000af6aaaf3ae2ecb0532,0.333333,0.666667,3.666667,140.0,348.500,208.500
1,001cca7ae9ae17fb1caed9dfb1094831,0.121827,0.527919,3.984772,3560.0,3868.003,308.003
2,002100f778ceb8431b7a1020ff7ab48f,0.134615,0.519231,3.903846,840.0,683.450,-156.550
